In [1]:
from IPython.core.display import display, HTML, Image
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
from collections import Counter
from itertools import chain
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer, TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
from itertools import chain
from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
from nltk.corpus import stopwords

In [5]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as SKLEARN_STOPWORDS
from spacy.lang.en.stop_words import STOP_WORDS as SPACY_STOPWORDS
from nltk.corpus import stopwords

SKLEARN_STOPWORDS = set(SKLEARN_STOPWORDS)
nltk.download('stopwords')
NLTK_STOPWORDS = set(stopwords.words('english'))
print(f'Loaded {len(NLTK_STOPWORDS)} stopwords from NLTK')
print(f'Loaded {len(SPACY_STOPWORDS)} stopwords from SPACY')
print(f'Loaded {len(SKLEARN_STOPWORDS)} stopwords from SKLEARN')
stop_words = list(set.union(*[SKLEARN_STOPWORDS, SPACY_STOPWORDS, NLTK_STOPWORDS]))
print('----------------------------------')
print(f'{len(stop_words)} combined stopwords')

Loaded 179 stopwords from NLTK
Loaded 326 stopwords from SPACY
Loaded 318 stopwords from SKLEARN
----------------------------------
409 combined stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
%%time
df = pd.read_parquet('../data/base.parquet')

CPU times: user 11.6 s, sys: 3.44 s, total: 15.1 s
Wall time: 10.8 s


In [7]:
tokenizer = TweetTokenizer()

In [ ]:
%%time
corpus = df['body'].values
corpus = [doc.lower() for doc in corpus]
corpus = [tokenizer.tokenize(doc) for doc in corpus]
corpus = [[token for token in doc if (not token.isnumeric() and len(token) > 1)] for doc in corpus]
corpus= [[token for token in doc if (not token in stop_words)] for doc in corpus]
bigram = Phrases(corpus, min_count=1)
trigram = Phrases(bigram[corpus], min_count=1)
fourgram = Phrases(trigram[corpus], min_count=1)
bigrams = []
trigrams = []
fourgrams = []
for doc in corpus:
    b = [b for b in bigram[doc] if b.count('_') == 1]
    t = [t for t in trigram[bigram[doc]] if t.count('_') == 2]
    f = [f for f in fourgram[trigram[bigram[doc]]] if f.count('_') == 3]
    bigrams.extend(b)
    trigrams.extend(t)
    fourgrams.extend(f)